In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

def pred_dataset(file_names, feature_set,random_state):
    source_path = 'C:/Kai_Zhang/MachineLearning/Unified gas Adsorption/CO2_adsorption/new_data'
    train_df = pd.DataFrame()
    test_df = pd.DataFrame()
    add_df = pd.DataFrame()
    for file_name in file_names:
        temp_data = pd.read_excel(os.path.join(source_path,file_name+'-02-02-2022.xlsx'),skiprows= 1 )
        temp_data = temp_data.dropna(axis=0,how = 'any',subset = feature_set)
        temp_data = temp_data[temp_data['Pressure']>0.01]
        index = list(set(temp_data['Index'].values))
        index = shuffle(index,random_state=random_state)
        print(len(index))
        test_add_index =  index[:int(len(index)*0.3)]
        #test_index= np.random.choice(index,int(0.1*len(index)),replace=False)
        test_index= test_add_index[:len(test_add_index)//2]
        add_index = test_add_index[len(test_add_index)//2:]
        train_x = temp_data.loc[~temp_data['Index'].isin( test_add_index)]

        test_x = temp_data.loc[temp_data['Index'].isin(test_index)]
        add_x = temp_data.loc[temp_data['Index'].isin(add_index)]
        
        train_df = pd.concat([train_df,train_x],axis=0)
        test_df = pd.concat([test_df,test_x],axis =0)
        add_df = pd.concat([add_df,add_x],axis =0)
        print(train_df.shape,test_df.shape,add_df.shape)
    return train_df,test_df,add_df

In [2]:

from sklearn.ensemble import GradientBoostingRegressor,\
    BaggingRegressor,ExtraTreesRegressor
from lightgbm import LGBMRegressor  

n_estimators = [50,100,120,150,180,200]
# define different models#,
models = [
    ("GBR",GradientBoostingRegressor(random_state=42)),\
    ('ETR',ExtraTreesRegressor(random_state=42,n_jobs=-1)),\
    ('LGBM',LGBMRegressor(n_jobs = -1,random_state = 42)),\
    #('BGLGBM',BaggingRegressor(LGBMRegressor(n_estimators = 200, n_jobs = -1,random_state = 42), random_state=42,n_jobs=-1)),\
    ]

# set search parameters grid for different models
para_grids = { 
    'GBR':{'n_estimators':n_estimators,'learning_rate':[0.1,0.5,1,2]},\
    'ETR':{'n_estimators':n_estimators},\
    'LGBM':{'num_leaves':[10,20,30,50],'learning_rate': [0.05,0.1,0.5,1],'n_estimators':n_estimators},\
    'BGLGBM':{'n_estimators':[10,30,50]},\
    }

In [3]:
from sklearn.model_selection import GridSearchCV,cross_validate,GroupKFold
from sklearn.ensemble import ExtraTreesRegressor
from  sklearn.metrics import mean_squared_error,r2_score
from sklearn.utils import shuffle

def model_CV(train_x,train_y,groups,model,para_grid):

    out_cv = GroupKFold(n_splits = 5)
    result = GridSearchCV(model,para_grid,cv= out_cv.get_n_splits(groups =groups),
    scoring='neg_mean_squared_error', return_train_score=True,n_jobs=-1)
    result.fit(train_x,train_y)
    
    model_refit =model.set_params(**result.best_params_)
    train_cv = cross_validate(model_refit,train_x,train_y,groups = groups,cv =out_cv,scoring = ('r2', 'neg_mean_squared_error'))
    train_mse_cv = -train_cv['test_neg_mean_squared_error'].mean()
    train_r2_cv = train_cv['test_r2'].mean()
    
    return [train_r2_cv,train_mse_cv],result.best_params_

# model evaluation
def model_eval(model,test_x,test_y):
      
    test_pre = model.predict(test_x)
    test_r2 = r2_score(test_pre,test_y)
    test_mse = mean_squared_error(test_y,test_pre)
    return test_r2,test_mse

# comparing different models
def model_comparison(model_list,para_grids,feature_list,gas_list):
    gas_list = gas_list 
    input_feature = feature_list
    output = ['Adsorp(mmol/g)']
    result_total = []

    for gas in gas_list:
        
            train_df_com = train_df[train_df['Label']==gas]
            test_df_com = test_df[test_df['Label']==gas]
            train_x = train_df_com[input_feature]
            test_x = test_df_com[input_feature]
            train_y = train_df_com[output].values
            test_y = test_df_com[output].values
            groups = train_df_com['Index']
            train_x, train_y, groups = shuffle(train_x, train_y, groups, random_state=42)
           
            for model_name, model in model_list:

                result, best_param = model_CV(train_x,train_y.squeeze(),groups,model,para_grids[model_name])
                model_refit = model.set_params(**best_param)
                model_refit.fit(train_x,train_y.squeeze())
                test_r2,test_mse = model_eval(model_refit,test_x,test_y.squeeze()) 
                result_total.append([gas,model_name+'_separate',result[0],result[1],-1,-1, test_r2,test_mse,best_param])
                
                print('Dataset {}, Algorithm {}, Test_r2 {}, Test_error {}'.format(gas,model_name+'_separate',test_r2,test_mse))

                
    return result_total

In [4]:
import numpy.random as nrd
from sklearn.decomposition import KernelPCA,SparsePCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from  collections import Counter

def bgk_pca(train_df,test_df): 
    #np.random.RandomState(42)# using major voting approach to find the outliers in the test dataset
    test_index = list(set(test_df["Index"].values))
    len_test = len(test_index)
    train_index = list(set(train_df["Index"].values))
    #total_feature = ["Index",'V','L','BET','Vt','Temp(K)']
    total_feature = ["Index",'BET','Vt','Temp(K)']
    #pca_feature =  ['V','L','BET','Vt','Temp(K)']
    pca_feature =  ['BET','Vt','Temp(K)']
    num_feature = len(pca_feature)
    removed_index = []
    res = []
    iters = 10
    multi_mse = []
    for i in range(iters):
        mses = []
        train_selected = train_df[train_df["Index"].isin(nrd.choice(train_index,int(len(train_index)*0.75),replace=False))] # modified here change fixed len to a the fraction of the len of the training dataset.
        data = pd.concat([test_df,train_selected])
        sub_data = data[total_feature].drop_duplicates()
        sub_data_scalered = MinMaxScaler().fit_transform(sub_data[pca_feature].values)
        
        """adding lines to determine the number of components to achieve 0.99 threshold"""
        kernel_pca = KernelPCA(kernel='poly',max_iter =100000,n_jobs =-1,gamma=1e-2,fit_inverse_transform=True,random_state=42)
        kpca_transform = kernel_pca.fit_transform(sub_data_scalered.reshape(num_feature,-1))
        explained_variance = np.var(kpca_transform, axis=0)
        explained_variance_ratio = explained_variance / np.sum(explained_variance)
        cumu_variance = np.cumsum(explained_variance_ratio)
        n_components = np.where(cumu_variance>0.99)[0][0]+1
        kernel_pca = KernelPCA(n_components=n_components,kernel='poly',max_iter =100000,n_jobs =-1,gamma=1e-2,fit_inverse_transform=True,random_state=42)
        """ The end of adding new lines"""

        sub_data_transformed = kernel_pca.fit_transform(sub_data_scalered.reshape(num_feature,-1))
        reconstructed = kernel_pca.inverse_transform(sub_data_transformed.reshape(num_feature,-1))
        for i in range(len(sub_data_scalered)):
            mses.append(mean_squared_error(sub_data_scalered[i],reconstructed.reshape(-1,num_feature)[i]))
            df_mse = pd.DataFrame(mses,columns = ['MSE'])
        df_mse['Indexs'] = sub_data["Index"].drop_duplicates().values
        mean_mse = df_mse["MSE"].mean()
        test_mse_df = df_mse[df_mse["Indexs"].isin(test_index)]
        outlier_index = test_mse_df[test_mse_df["MSE"]>3*mean_mse]["Indexs"].values.tolist()
        removed_index.extend(outlier_index)
        multi_mse.append(mean_mse)

    counter = Counter(removed_index)
    thresh = int(0.7*iters)
    for key,values in counter.most_common():
        if values>=thresh:
            res.append(key)
            
        if values<thresh: break

    return np.mean(multi_mse)#res

In [12]:
import os
from tqdm import tqdm
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
combin_5 = ['Vt',"Vmic",'Vmeso',]
feature_list = [base_feature+combin_5+condition_feature]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param','kpg_pca_mse']
#file_name = ['Total',"Meso","Micro",'All','Vmic_meso']
file_name = ['BET_plus_Vt']
feature_set = ["Vt","Vmic",'Vmeso',]
gas_list = ['CO2']
recons_error = []
total_results = []
label = ['Without new adding','With new adding']
for i in tqdm(range(50)):
    train_dfs,test_df,add_df = pred_dataset(gas_list,feature_set,random_state=i)#
    for i in range(2):
        if not i:
            select_list = list(set(add_df['Index'].values))[:len(set(add_df['Index'].values))//2]
            print(len(select_list))
            add_df = add_df[add_df['Index'].isin(select_list)]
            train_df = pd.concat([train_dfs,add_df],axis= 0)
        else:
            train_df = train_dfs

        k_recon_mse = bgk_pca(train_df,test_df)
        results = model_comparison(models,para_grids, feature_list[0],gas_list)
        temp_results  = []
        for ele in results:
            temp_results.append(ele+[k_recon_mse])
        print("Datasetstype: {}, recons_mse: {:.4f}".format(label[i],k_recon_mse))
        total_results.extend(temp_results)
files_name = 'Res_pca_The_impact_of_different_training_sample_size_of_CO2_add_half_'+file_name[0]+'_result_'+str(i)+'.csv'
pd.DataFrame(total_results,columns = columns).to_csv(os.path.join('./3_The_impact_of_different_training_sample_size',files_name)) 

  0%|          | 0/50 [00:00<?, ?it/s]

2028
(15061, 27) (3260, 27) (3260, 27)
152
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9536452508770378, Test_error 0.6294428439161793
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9439560756245216, Test_error 0.7632598455806767
Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9500558755264121, Test_error 0.6510087162556933
Datasetstype: Without new adding, recons_mse: 0.0194
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9441972552178082, Test_error 0.7382289479314933
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9417422199288029, Test_error 0.8039262751125303


  2%|▏         | 1/50 [02:52<2:21:12, 172.92s/it]

Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9414345446055676, Test_error 0.7657783978721063
Datasetstype: With new adding, recons_mse: 0.0199
2028
(15199, 27) (3159, 27) (3223, 27)
152
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9437450622318072, Test_error 0.9264738316984321
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9412930276751457, Test_error 0.9184855012456189
Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9475723944717708, Test_error 0.8187582006434645
Datasetstype: Without new adding, recons_mse: 0.0189
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9363764447953846, Test_error 1.0316716017820198
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9394131096587868, Test_error 0.9337771640765642


  4%|▍         | 2/50 [05:44<2:17:38, 172.05s/it]

...

Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9531509094769183, Test_error 0.6721588438507464
Datasetstype: With new adding, recons_mse: 0.0195
2028
(15093, 27) (3200, 27) (3288, 27)
152
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9461588679112173, Test_error 0.5299149258434634
Dataset CO2, Algorithm ETR_separate, Test_r2 0.935077407793816, Test_error 0.5724320972970232
Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9463217387443258, Test_error 0.48593756491648477
Datasetstype: Without new adding, recons_mse: 0.0193
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9354806300269168, Test_error 0.5887911699087254
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9302429525442878, Test_error 0.6116402541194842


 98%|█████████▊| 49/50 [2:17:10<02:46, 166.71s/it]

Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9418687015597208, Test_error 0.5244502463082326
Datasetstype: With new adding, recons_mse: 0.0208
2028
(15216, 27) (3177, 27) (3188, 27)
152
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9372199821261243, Test_error 0.6826981490443373
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9359804904718263, Test_error 0.7163215730787867
Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9360416819819383, Test_error 0.6705506621149924
Datasetstype: Without new adding, recons_mse: 0.0189
Dataset CO2, Algorithm GBR_separate, Test_r2 0.9335432993506995, Test_error 0.742206644269408
Dataset CO2, Algorithm ETR_separate, Test_r2 0.9344379151973488, Test_error 0.7440589570543991


100%|██████████| 50/50 [2:19:59<00:00, 167.99s/it]

Dataset CO2, Algorithm LGBM_separate, Test_r2 0.9426239926719948, Test_error 0.6531438542172406
Datasetstype: With new adding, recons_mse: 0.0200


In [14]:
add_df.head(5)

,Unnamed: 0,Literature,Unnamed: 1,GAC,GAS,E,S,A,B,V,...,%N,Temp(K),Pressure,Adsorp(mmol/g),Index,logP,logQ,logD,Label,Vmic_index
255,255,NaN,255,UC-700-4,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,0.29,298.0,0.050968,0.261194,14,-2.976567,-1.342492,1.634075,CO2,1
256,256,NaN,256,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,0.29,298.0,0.086415,0.417910,14,-2.448591,-0.872488,1.576103,CO2,1
257,257,NaN,257,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,0.29,298.0,0.130724,0.609453,14,-2.034670,-0.495194,1.539476,CO2,1
258,258,NaN,258,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,0.29,298.0,0.183898,0.853234,14,-1.693374,-0.158722,1.534652,CO2,1
259,259,NaN,259,NaN,CarbonDiox,0,0.28,0.05,0.1,0.2809,...,0.29,298.0,0.228190,0.992537,14,-1.477578,-0.007491,1.470087,CO2,1


In [6]:
import pandas as pd
#file_name = ['BET_only','BET_plut_Vt',"BET_Vmic","BET_Vmeso",'BET_Vt_Vmeso','BET_Vt_Vmic',"BET_Vt_Vmic_meso","BET_Vmic_meso"]
file_name = ['BET_plus_Vt']
cal_columns= ["CV_r2","CV_mse","test_r2_separa_model","test_mse_separa_model",'kpg_pca_mse']
tpd = []
gas_list = ['CO2']
for gas in gas_list:
    
    #fraction = range(50,len(list(set(train_dfs["Index"].values))),interval)
    fraction = range(50,2125,50)
    for k in fraction: 
        df_list = []
        for i in range(45):
            #for j in range(len(feature_list)):
                files_name = 'Res_pca_The_impact_of_different_training_sample_size_of_'+str(k)+"_"+file_name[0]+'_result_'+str(i)+'.csv'
                df_list.append(pd.read_csv(os.path.join('./3_The_impact_of_different_training_sample_size',files_name))[cal_columns] )
        pd.concat(df_list).groupby(level=0).mean().to_csv(os.path.join('./3_The_impact_of_different_training_sample_size',   str(k)+'-mean.csv'))
        pd.concat(df_list).groupby(level=0).std().to_csv(os.path.join('./3_The_impact_of_different_training_sample_size',str(k)+'-std.csv'))
        tpd = df_list

Post-post treatment

In [8]:
file_path = "./3_The_impact_of_different_training_sample_size" #/Methane-25(test)-775(max training)-with-reconstruction-mse
mean_df = pd.DataFrame()
std_df = pd.DataFrame()
for i in range(50,2125,50):#range(50,len(list(set(train_dfs["Index"].values))),interval):
    mean_temp = pd.read_csv(os.path.join(file_path,str(i)+'-mean.csv'))
    std_temp = pd.read_csv(os.path.join(file_path,str(i)+'-std.csv'))
    mean_df = pd.concat([mean_df,mean_temp],axis = 0)
    std_df = pd.concat([std_df,std_temp],axis = 0)
mean_df.to_csv(os.path.join(file_path,"Total_mean.csv"))
std_df.to_csv(os.path.join(file_path,"Total_std.csv"))


Only calculating the reconstruction error

In [77]:
import os
base_feature = ['V','S','L','BET',]
condition_feature = ['Temp(K)','Pressure']
combin_1 = ['Vt']
combin_2 = ["Vmeso"]
combin_3 = ['Vmic']
combin_4 = ['Vt',"Vmeso",]
combin_3 = ['Vt',"Vmic",] 
combin_5 = ['Vt',"Vmic",'Vmeso',]
combin_6 = ["Vmic",'Vmeso',]
feature_list = [base_feature+combin_1+condition_feature]
columns = ['Gas','Model_name','CV_r2','CV_mse','test_r2_total_model','test_mse_by_total_model','test_r2_separa_model','test_mse_separa_model','best_param','kpg_pca_mse']
#file_name = ['Total',"Meso","Micro",'All','Vmic_meso']
file_name = ['BET_plus_Vt']
feature_set = ["Vt",]
gas_list = ['CO2']

for i in range(15):
    recons_error = []
    train_index = []
    train_dfs,test_df = pred_dataset(gas_list,feature_set)#
    nums_test = len(list(set(test_df["Index"].values)))
    
    index = list(set(train_dfs['Index'].values))
    fraction = range(100,len(list(set(train_dfs["Index"].values))),50)
    for k in range(len(fraction)):
        print(k)
        
        #k = 50
        selected_index = np.random.choice(index,50,replace=False)
        
        for ele in selected_index:
            index.remove(ele)
        train_index.extend(selected_index) 
        print(k, len(train_index))
        train_df = temp_df.loc[temp_df['Index'].isin(train_index)]
        k_recon_mse = bgk_pca(train_df,test_df)
        recons_error.append(k_recon_mse)
        files_name = 'Res_pca_The_impact_of_different_training_sample_size_of_'+str(50*k)+"_"+file_name[0]+'_result_'+str(i)+'.csv'
        pd.DataFrame(recons_error).to_csv(os.path.join('./3_The_impact_of_different_training_sample_size',files_name)) 

2145
0
0 50
1
1 100
2
2 150
3
3 200
4
4 250
5
5 300
6
6 350
7
7 400
8
8 450
9
9 500
10
10 550
11
11 600
12
12 650
13
13 700
14
14 750
15
15 800
16
16 850
17
17 900
18
18 950
19
19 1000
20
20 1050
21
21 1100
22
22 1150
23
23 1200
24
24 1250
25
25 1300
26
26 1350
27
27 1400
28
28 1450
29
29 1500
30
30 1550
31
31 1600
32
32 1650
33
33 1700
34
34 1750
35
35 1800
36
36 1850
37
37 1900
38
38 1950
39
39 2000
2145
0
0 50
1
1 100
2
2 150
3
3 200
4
4 250
5
5 300
6
6 350
7
7 400
8
8 450
9
9 500
10
10 550
11
11 600
12
12 650
13
13 700
14
14 750
15
15 800
16
16 850
17
17 900
18
18 950
19
19 1000
20
20 1050
21
21 1100
22
22 1150
23
23 1200
24
24 1250
25
25 1300
26
26 1350
27
27 1400
28
28 1450
29
29 1500
30
30 1550
31
31 1600
32
32 1650
33
33 1700
34
34 1750
35
35 1800
36
36 1850
37
37 1900
38
38 1950
39
39 2000
2145
0
0 50
1
1 100
2
2 150
3
3 200
4
4 250
5
5 300
6
6 350
7
7 400
8
8 450
9
9 500
10
10 550
11
11 600
12
12 650
13
13 700
14
14 750
15
15 800
16
16 850
17
17 900
18
18 950
19
19 1000
20
20

In [73]:
len(range(2))

2